In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import lightgbm_with_simple_features as lgbmsf
import json

with open('config.json', 'r') as f:
    config = json.load(f)
    
NUM_ROWS = config["NUM_ROWS"]

In [2]:
!ls ./data

HomeCredit_columns_description.csv
POS_CASH_balance.csv
Projet+Mise+en+prod+-+home-credit-default-risk.zip
application_test.csv
application_train.csv
bureau.csv
bureau_balance.csv
credit_card_balance.csv
installments_payments.csv
previous_application.csv
sample_submission.csv


### Inspection du kernel

### application_train.csv
- This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
- Static data for all applications. One row represents one loan in our data sample.

**Observations**

- les variables _DAYS_EMPLOYED_ et _DAYS_BIRTH_ sont négatives - une fois _DAYS_EMPLOYED_ nettoyée de ses valeurs aberrantes.

In [6]:
application_train_df = pd.read_csv('./data/application_train.csv')
application_train_df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
train_df_processed = lgbmsf.application_train_test(num_rows=NUM_ROWS)
train_df_processed.head()

Train samples: 1000


,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,100002,1,0,0,0,0,202500.0,406597.5,24700.5,351000.0,...,0,1,0,1,0,0.067329,0.498036,202500.0,0.121978,0.060749
1,100003,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,1,0,0.070862,0.208736,135000.0,0.132217,0.027598
2,100004,0,0,1,0,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0.011814,0.500000,67500.0,0.100000,0.050000
3,100006,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,0,0.159905,0.431748,67500.0,0.219900,0.094941
4,100007,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0.152418,0.236842,121500.0,0.179963,0.042623


### bureau and bureau_balance

In [4]:
bb_df = lgbmsf.bureau_and_balance(num_rows=NUM_ROWS)

In [5]:
bb_df.head()

,BURO_DAYS_CREDIT_MIN,BURO_DAYS_CREDIT_MAX,BURO_DAYS_CREDIT_MEAN,BURO_DAYS_CREDIT_VAR,BURO_DAYS_CREDIT_ENDDATE_MIN,BURO_DAYS_CREDIT_ENDDATE_MAX,BURO_DAYS_CREDIT_ENDDATE_MEAN,BURO_DAYS_CREDIT_UPDATE_MEAN,BURO_CREDIT_DAY_OVERDUE_MAX,BURO_CREDIT_DAY_OVERDUE_MEAN,...,CLOSED_AMT_CREDIT_SUM_OVERDUE_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_MEAN,CLOSED_AMT_CREDIT_SUM_LIMIT_SUM,CLOSED_AMT_ANNUITY_MAX,CLOSED_AMT_ANNUITY_MEAN,CLOSED_CNT_CREDIT_PROLONG_SUM,CLOSED_MONTHS_BALANCE_MIN_MIN,CLOSED_MONTHS_BALANCE_MAX_MAX,CLOSED_MONTHS_BALANCE_SIZE_MEAN,CLOSED_MONTHS_BALANCE_SIZE_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
101060,-2632,-709,-1328.500000,779007.000000,-2478.0,783.0,-522.5,-820.750000,0,0.0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
104261,-587,-587,-587.000000,NaN,872.0,872.0,872.0,-6.000000,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115001,-2784,-174,-847.625000,745005.410714,-2387.0,3198.0,-57.0,-623.875000,0,0.0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0
118247,-2022,-484,-1206.714286,333125.904762,-1838.0,-182.0,-1092.0,-508.714286,0,0.0,...,0.0,0.0,0.0,NaN,NaN,2.0,NaN,NaN,NaN,0.0
119306,-1040,-243,-643.666667,158816.333333,-917.0,1090.0,207.0,-421.666667,0,0.0,...,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0


## previous_applications

In [7]:
previous_app_df = lgbmsf.previous_applications(num_rows=NUM_ROWS)
previous_app_df.head()

,PREV_AMT_ANNUITY_MIN,PREV_AMT_ANNUITY_MAX,PREV_AMT_ANNUITY_MEAN,PREV_AMT_APPLICATION_MIN,PREV_AMT_APPLICATION_MAX,PREV_AMT_APPLICATION_MEAN,PREV_AMT_CREDIT_MIN,PREV_AMT_CREDIT_MAX,PREV_AMT_CREDIT_MEAN,PREV_APP_CREDIT_PERC_MIN,...,REFUSED_HOUR_APPR_PROCESS_START_MAX,REFUSED_HOUR_APPR_PROCESS_START_MEAN,REFUSED_RATE_DOWN_PAYMENT_MIN,REFUSED_RATE_DOWN_PAYMENT_MAX,REFUSED_RATE_DOWN_PAYMENT_MEAN,REFUSED_DAYS_DECISION_MIN,REFUSED_DAYS_DECISION_MAX,REFUSED_DAYS_DECISION_MEAN,REFUSED_CNT_PAYMENT_MEAN,REFUSED_CNT_PAYMENT_SUM
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100077,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,14.0,14.0,NaN,NaN,NaN,-217.0,-217.0,-217.0,NaN,0.0
100373,10872.990,10872.990,10872.990,115695.0,115695.0,115695.0,115119.0,115119.0,115119.0,1.005004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101011,6856.065,6856.065,6856.065,65866.5,65866.5,65866.5,66735.0,66735.0,66735.0,0.986986,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101167,5437.530,5437.530,5437.530,120060.0,120060.0,120060.0,120060.0,120060.0,120060.0,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101211,4172.940,4172.940,4172.940,29205.0,29205.0,29205.0,21204.0,21204.0,21204.0,1.377334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## pos_cash

In [8]:
poc_cash = lgbmsf.pos_cash(num_rows=NUM_ROWS)
poc_cash.head()

,POS_MONTHS_BALANCE_MAX,POS_MONTHS_BALANCE_MEAN,POS_MONTHS_BALANCE_SIZE,POS_SK_DPD_MAX,POS_SK_DPD_MEAN,POS_SK_DPD_DEF_MAX,POS_SK_DPD_DEF_MEAN,POS_NAME_CONTRACT_STATUS_Active_MEAN,POS_NAME_CONTRACT_STATUS_Completed_MEAN,POS_NAME_CONTRACT_STATUS_Signed_MEAN,POS_NAME_CONTRACT_STATUS_nan_MEAN,POS_COUNT
SK_ID_CURR,,,,,,,,,,,,
100187,-43,-43.0,1,0,0,0,0,1.0,0.0,0,0,1
100457,-16,-16.0,1,0,0,0,0,1.0,0.0,0,0,1
100676,-41,-41.0,1,0,0,0,0,1.0,0.0,0,0,1
101741,-33,-33.0,1,0,0,0,0,1.0,0.0,0,0,1
102489,-18,-18.0,1,0,0,0,0,1.0,0.0,0,0,1


## installment_payments

In [9]:
installment_p_df = lgbmsf.installments_payments(num_rows=NUM_ROWS)
installment_p_df.head()

,INSTAL_NUM_INSTALMENT_VERSION_NUNIQUE,INSTAL_DPD_MAX,INSTAL_DPD_MEAN,INSTAL_DPD_SUM,INSTAL_DBD_MAX,INSTAL_DBD_MEAN,INSTAL_DBD_SUM,INSTAL_PAYMENT_PERC_MAX,INSTAL_PAYMENT_PERC_MEAN,INSTAL_PAYMENT_PERC_SUM,...,INSTAL_AMT_INSTALMENT_MEAN,INSTAL_AMT_INSTALMENT_SUM,INSTAL_AMT_PAYMENT_MIN,INSTAL_AMT_PAYMENT_MAX,INSTAL_AMT_PAYMENT_MEAN,INSTAL_AMT_PAYMENT_SUM,INSTAL_DAYS_ENTRY_PAYMENT_MAX,INSTAL_DAYS_ENTRY_PAYMENT_MEAN,INSTAL_DAYS_ENTRY_PAYMENT_SUM,INSTAL_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100012,1,0.0,0.0,0.0,41.0,41.0,41.0,1.000000,1.000000,1.000000,...,5242.860,5242.860,5242.860,5242.860,5242.860,5242.860,-428.0,-428.0,-428.0,1
100187,1,0.0,0.0,0.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,...,10281.870,10281.870,10281.870,10281.870,10281.870,10281.870,-1071.0,-1071.0,-1071.0,1
100193,1,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,...,301.860,301.860,301.860,301.860,301.860,301.860,-1153.0,-1153.0,-1153.0,1
100266,1,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,...,3.825,3.825,3.825,3.825,3.825,3.825,-58.0,-58.0,-58.0,1
100299,1,30.0,30.0,30.0,0.0,0.0,0.0,0.000003,0.000003,0.000003,...,15737.985,15737.985,0.045,0.045,0.045,0.045,-2293.0,-2293.0,-2293.0,1


## credit_card balance

In [10]:
cc_balance = lgbmsf.credit_card_balance(num_rows=NUM_ROWS)
cc_balance.head()

,CC_MONTHS_BALANCE_MIN,CC_MONTHS_BALANCE_MAX,CC_MONTHS_BALANCE_MEAN,CC_MONTHS_BALANCE_SUM,CC_MONTHS_BALANCE_VAR,CC_AMT_BALANCE_MIN,CC_AMT_BALANCE_MAX,CC_AMT_BALANCE_MEAN,CC_AMT_BALANCE_SUM,CC_AMT_BALANCE_VAR,...,CC_NAME_CONTRACT_STATUS_Completed_MAX,CC_NAME_CONTRACT_STATUS_Completed_MEAN,CC_NAME_CONTRACT_STATUS_Completed_SUM,CC_NAME_CONTRACT_STATUS_Completed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100218,-4,-4,-4,-4,NaN,748180.935,748180.935,748180.935,748180.935,NaN,...,0,0,0,NaN,0,0,0,0,NaN,1
100662,-5,-5,-5,-5,NaN,0.000,0.000,0.000,0.000,NaN,...,0,0,0,NaN,0,0,0,0,NaN,1
100796,-4,-4,-4,-4,NaN,0.000,0.000,0.000,0.000,NaN,...,0,0,0,NaN,0,0,0,0,NaN,1
101399,-7,-7,-7,-7,NaN,410062.365,410062.365,410062.365,410062.365,NaN,...,0,0,0,NaN,0,0,0,0,NaN,1
101827,-6,-6,-6,-6,NaN,0.000,0.000,0.000,0.000,NaN,...,0,0,0,NaN,0,0,0,0,NaN,1


## sample_submission

In [12]:
results = pd.read_csv('./data/sample_submission.csv')
results.head()

,SK_ID_CURR,TARGET
0,100001,0.5
1,100005,0.5
2,100013,0.5
3,100028,0.5
4,100038,0.5


Ok.

### Definitions des tables

**HomeCredit_columns_description.csv**

In [11]:
application_train = pd.read_csv('./data/HomeCredit_columns_description.csv')
HomeCredit_df.drop

,0,Table,Row,Description,Special
0,1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
1,2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
2,5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
3,6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
4,7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN


## Classes déséquilibrées

In [2]:
# Construction du dataframe "global"
data = lgbmsf.join_df(NUM_ROWS)
data.info()

Train samples: 10000
Bureau df shape: (2011, 108)
Process bureau and bureau_balance - done in 1s
Previous applications df shape: (9734, 242)
Process previous_applications - done in 2s
Pos-cash balance df shape: (9494, 15)
Process POS-CASH balance - done in 0s
Installments payments df shape: (8893, 26)
Process installments payments - done in 1s
Credit card balance df shape: (9520, 131)
Process credit card balance - done in 1s
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Columns: 765 entries, SK_ID_CURR to CC_COUNT
dtypes: float64(593), int64(14), uint8(158)
memory usage: 47.9 MB


## Target

In [3]:
data.TARGET.unique()

array([1, 0])

In [4]:
data['TARGET'].value_counts()

0    9225
1     775
Name: TARGET, dtype: int64

Le jeu de données est donc bien déséquilibré

Essayer de proposer un premier modèle - baseline: pourquoi pas une simpe regression logistique?
Il faudra diminuer le nombre de variables en entrée: on a 765 colonnes
--> compliqué à cause du mix variables continues et variables catégorielles.
Combien de variables catégorielles?


In [3]:
int_cols = [col for col in data.columns if data[col].dtype=='int64']
data[int_cols].head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,DAYS_BIRTH,DAYS_ID_PUBLISH,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY
0,100002,1,0,-9461,-2120,2,2,10,0,0,0,0,0,0
1,100003,0,0,-16765,-291,1,1,11,0,0,0,0,0,0
2,100004,0,0,-19046,-2531,2,2,9,0,0,0,0,0,0
3,100006,0,0,-19005,-2437,2,2,17,0,0,0,0,0,0
4,100007,0,0,-19932,-3458,2,2,11,0,0,0,0,1,1


In [21]:
uint_cols = [col for col in data.columns if data[col].dtype=='uint8']
data[uint_cols].head()

,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
1,1,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


1

GridSearchCV pour optimiser le paramètre de régularisation.